In [7]:
import pandas as pd
import pickle

#### Read Data In

In [32]:
import os
import re

path = "Android/incoming/"
filelist = {}

def insertIntoDataStruct(day, f, aDict):
    if not day in aDict:
        aDict[day] = [f]
    else:
        aDict[day].append(f)

for f in os.listdir(path):
    if f[0] != '.' and 'LABELED' not in str(f):
        if re.search("[^_]*-[^_]*", f):
            day = re.search("[^_]*-[^_]*", f).group()
            insertIntoDataStruct(day, f, filelist)

#### Necessary Input

In [58]:
andrew_events = pickle.load(open("andrew_commute_labels.p", "rb" ))

In [59]:
days_and_starts = {'2017-03-19':['10:10:01', '18:12:00'], '2017-03-20': ['08:26:00', '21:12:00'], 
                  '2017-03-21': ['08:49:00', '17:39:00'], '2017-03-22': ['08:51:00', '20:24:02'],
                  '2017-03-23': ['08:52:30'], '2017-03-26': ['10:42:00', '17:33:30'], 
                  '2017-03-27': ['08:19:30']}

#### Functions for Classifying

In [72]:
def add_acceleration_magnitude(df):
    "Adds Acceleration magnitude to a dataframe. First 3 columns must be acceleration vectors. Must be 3D."
    from math import sqrt

    accel_mag = []
    
    for row in range(len(df)):
        accel_mag.append(sqrt(df.iloc[row, 0]**2 + df.iloc[row, 1]**2 + df.iloc[row, 2]**2))

    df['accel_mag'] = accel_mag
    return 'finished!'

In [73]:
def rel_time_to_real_time(df, day, start_time):
    from pandas.tseries.offsets import DateOffset
    df.index = [pd.Timestamp(day + " " + start_time) + DateOffset(seconds=x) for x in df.index]
    return "Finished!"

In [74]:
def add_classifications(df, day, events):
    '''Pass events as a list of tuples (datetime, eventcode) to classify different events '''
    from pandas.tseries.offsets import DateOffset
    
    for event in events:
        df.loc[day + " " + event[0], 'classification'] = event[1]
        if event[1] == "T_S":
            df.loc[str((pd.Timestamp(day + " " + event[0])+ pd.DateOffset(seconds=30))), 'classification'] = "T_D"
    df = df.fillna(method='ffill', inplace = True)
    
    return "Finished!"

In [75]:
def label_raw_android(f, day, events, start_time, accel_mag = False):
    df = pd.read_csv((path + f), index_col = 0)
    if accel_mag == True:
        add_acceleration_magnitude(df)
    rel_time_to_real_time(df, day, start_time)
    df['classification'] = None
    add_classifications(df, day, events)
    df.to_csv("Android/classified/labeled_" + f)
    print "Finished!"

In [76]:
days_and_starts

{'2017-03-19': ['10:10:01', '18:12:00'],
 '2017-03-20': ['08:26:00', '21:12:00'],
 '2017-03-21': ['08:49:00', '17:39:00'],
 '2017-03-22': ['08:51:00', '20:24:02'],
 '2017-03-23': ['08:52:30'],
 '2017-03-26': ['10:42:00', '17:33:30'],
 '2017-03-27': ['08:19:30']}

#### Classifying Data

In [77]:
#Use this function to Classify an android file. Must be done manually now... will automate
#later if we decide to use a lot of android data 
label_raw_android("Andrew_03-19-2017_Morning_acc.csv", "2017-03-19", andrew_events['2017-3-19'], "10:10:01")

TypeError: 'NoneType' object is not iterable